<a href="https://colab.research.google.com/github/Vyoma-garg/Natural-Language-Processing/blob/main/Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Getting dataset**

In [ ]:
import pandas as pd
import gzip
false=False
true=True

def parse(path):
  g = gzip.open('/content/Magazine_Subscriptions.json.gz', 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Magazine_Subscriptions.json.gz')

Total Number of Magazine Subscriptions reviews 

In [ ]:
len(df)

89689

Total Number of reviews taken in consideration 

In [ ]:
df=df[:25000]
len(df)

25000

In [ ]:
df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,Sentiment,Binary_Sentiment
0,5.0,9,False,"11 8, 2001",AH2IFH762VY5U,B00005N7P0,ted sedlmayr,"for computer enthusiast, MaxPC is a welcome si...","AVID READER SINCE ""boot"" WAS THE NAME",1005177600,NaN,NaN,Positive,1.0
1,5.0,9,False,"10 31, 2001",AOSFI0JEYU4XM,B00005N7P0,Amazon Customer,Thank god this is not a Ziff Davis publication...,The straight scoop,1004486400,NaN,NaN,Positive,1.0
2,3.0,14,False,"03 24, 2007",A3JPFWKS83R49V,B00005N7OJ,Bryan Carey,Antiques Magazine is a publication made for an...,"Antiques Magazine is Good, but not for Everyone",1174694400,{'Format:': ' Print Magazine'},NaN,Negative,0.0
3,5.0,13,False,"11 10, 2006",A19FKU6JZQ2ECJ,B00005N7OJ,Patricia L. Porada,This beautiful magazine is in itself a work of...,THE DISCERNING READER,1163116800,{'Format:': ' Print Magazine'},NaN,Positive,1.0
4,5.0,NaN,True,"07 14, 2014",A25MDGOMZ2GALN,B00005N7P0,Alvey,A great read every issue.,Five Stars,1405296000,NaN,NaN,Positive,1.0


Adding a column for Positive and Negative Sentiment

In [ ]:
df.loc[df['overall']<=3, 'Sentiment']='Negative'
df.loc[df['overall']>3, 'Sentiment']='Positive'
df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,Sentiment
0,5.0,9,False,"11 8, 2001",AH2IFH762VY5U,B00005N7P0,ted sedlmayr,"for computer enthusiast, MaxPC is a welcome si...","AVID READER SINCE ""boot"" WAS THE NAME",1005177600,NaN,NaN,Positive
1,5.0,9,False,"10 31, 2001",AOSFI0JEYU4XM,B00005N7P0,Amazon Customer,Thank god this is not a Ziff Davis publication...,The straight scoop,1004486400,NaN,NaN,Positive
2,3.0,14,False,"03 24, 2007",A3JPFWKS83R49V,B00005N7OJ,Bryan Carey,Antiques Magazine is a publication made for an...,"Antiques Magazine is Good, but not for Everyone",1174694400,{'Format:': ' Print Magazine'},NaN,Negative
3,5.0,13,False,"11 10, 2006",A19FKU6JZQ2ECJ,B00005N7OJ,Patricia L. Porada,This beautiful magazine is in itself a work of...,THE DISCERNING READER,1163116800,{'Format:': ' Print Magazine'},NaN,Positive
4,5.0,NaN,True,"07 14, 2014",A25MDGOMZ2GALN,B00005N7P0,Alvey,A great read every issue.,Five Stars,1405296000,NaN,NaN,Positive


Adding a column for Sentiments in binary form

In [ ]:
df.loc[df['Sentiment']=='Negative', 'Binary_Sentiment']=0
df.loc[df['Sentiment']=='Positive', 'Binary_Sentiment']=1
df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,Sentiment,Binary_Sentiment
0,5.0,9,False,"11 8, 2001",AH2IFH762VY5U,B00005N7P0,ted sedlmayr,"for computer enthusiast, MaxPC is a welcome si...","AVID READER SINCE ""boot"" WAS THE NAME",1005177600,NaN,NaN,Positive,1.0
1,5.0,9,False,"10 31, 2001",AOSFI0JEYU4XM,B00005N7P0,Amazon Customer,Thank god this is not a Ziff Davis publication...,The straight scoop,1004486400,NaN,NaN,Positive,1.0
2,3.0,14,False,"03 24, 2007",A3JPFWKS83R49V,B00005N7OJ,Bryan Carey,Antiques Magazine is a publication made for an...,"Antiques Magazine is Good, but not for Everyone",1174694400,{'Format:': ' Print Magazine'},NaN,Negative,0.0
3,5.0,13,False,"11 10, 2006",A19FKU6JZQ2ECJ,B00005N7OJ,Patricia L. Porada,This beautiful magazine is in itself a work of...,THE DISCERNING READER,1163116800,{'Format:': ' Print Magazine'},NaN,Positive,1.0
4,5.0,NaN,True,"07 14, 2014",A25MDGOMZ2GALN,B00005N7P0,Alvey,A great read every issue.,Five Stars,1405296000,NaN,NaN,Positive,1.0


In [ ]:
df.tail()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,Sentiment,Binary_Sentiment
24995,5.0,NaN,True,"03 24, 2015",A1AG7VJ1T35X77,B00005QJE0,Darrin R. West,Gorgeous large format magazine. Very inspiring...,Gorgeous and Inspiring.,1427155200,{'Format:': ' Print Magazine'},NaN,Positive,1.0
24996,4.0,NaN,True,"07 30, 2014",A365DG6L5EUQQB,B00005QJE0,Charles Belew,Very happy with our subscribtions!,Four Stars,1406678400,{'Format:': ' Print Magazine'},NaN,Positive,1.0
24997,5.0,NaN,True,"02 26, 2014",A312FAW297PIOZ,B00005QJE0,kwilson,Love advice on gardening and the different gar...,Would. recommend this magazine to everyone .,1393372800,{'Format:': ' Print Magazine'},NaN,Positive,1.0
24998,5.0,NaN,True,"04 4, 2013",AUHOX5LLRIF11,B00005QJE0,Amazon Customer,photos are so good I look at these mags over a...,best gardening magazine,1365033600,{'Format:': ' Print Magazine'},NaN,Positive,1.0
24999,5.0,4,True,"02 6, 2013",A2P4CPM9S3QBDU,B00005QJE0,Linda W Owen,The English Garden is a wonderful gardening ma...,"The English Garden, good magizine for Americans",1360108800,{'Format:': ' Print Magazine'},NaN,Positive,1.0


In [ ]:
corpus = df["reviewText"].fillna(' ').values.tolist()
rating=df['overall'].values.tolist()
sentiment=df['Sentiment'].values.tolist()
bin_sentiment=df['Binary_Sentiment'].values.tolist()
print(bin_sentiment)

**1. (a) CountVectorizer**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 

# Create a Vectorizer Object 
vectorizer = CountVectorizer(lowercase=True, stop_words=stopset,) 

#tokenize and build vocab 
X_cv=vectorizer.fit_transform(corpus)  #25000x25711
print(X_cv[0])
print(vectorizer.get_feature_names())

# Printing the identified Unique words along with their indices 
print("Vocabulary: ", vectorizer.vocabulary_)

data = vectorizer.vocabulary_

#pd.Series().to_frame(vectorizer.vocabulary)
dic= pd.DataFrame.from_dict([data])
dic

## **Module Topic Modelling**

In [ ]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

In [ ]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=5, # Number of topics
                                      max_iter=10,               
# Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          
# Random state
                                      batch_size=128,            
# n docs in each learning iter
                                      evaluate_every = -1,       
# compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               
# Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(X_cv)
print(lda_model)  # Model attributes


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [ ]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(X_cv))
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(X_cv))
# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -4576154.315001247
Perplexity:  2292.9264546550253
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 5,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [ ]:
# Create Document — Topic Matrix
lda_output = lda_model.transform(X_cv)

# column names
topicnames = ['Topic' + str(i) for i in range(lda_model.n_components)]

# index names
docnames = ['Doc' + str(i) for i in range(len(corpus))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
 color = 'green' if val > .1 else 'black'
 return 'color: {col}'.format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.100000,0.160000,0.290000,0.070000,0.380000,4
Doc1,0.010000,0.220000,0.510000,0.010000,0.250000,2
Doc2,0.010000,0.590000,0.000000,0.400000,0.000000,1
Doc3,0.010000,0.450000,0.010000,0.530000,0.010000,3
Doc4,0.040000,0.840000,0.040000,0.040000,0.040000,1
Doc5,0.010000,0.940000,0.010000,0.010000,0.010000,1
Doc6,0.010000,0.200000,0.690000,0.000000,0.090000,2
Doc7,0.020000,0.220000,0.020000,0.020000,0.730000,4
Doc8,0.020000,0.480000,0.020000,0.190000,0.300000,1
Doc9,0.010000,0.330000,0.230000,0.420000,0.010000,3


In [ ]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

,00,000,0000000001,00005,0006058434,00s,01,0195366824,02,02081,0235,02809,03,0320,0399146113,04,0465061796,05,05465,0595472273,06,07,0714838284,0736063684,08,0801,0812972821,0812976665,09,10,100,1000,100000,1000f,100k,100x,101,102,1022,103,...,zero,zeroes,zeroing,zest,zeta,zetia,ziff,zillion,zillions,zim,zimbabwe,zinczenko,zine,zines,zinio,zion,zip,zipped,zipusa,zizzy,zombie,zombies,zondervan,zone,zones,zoo,zooey,zoolander,zoology,zoom,zoomed,zooming,zoon,zp,zsa,zucchini,zune,zurich,zx,zzz
Topic0,0.203604,48.782457,0.202138,0.201355,0.832754,0.215877,0.203223,0.885570,0.202492,1.029897,0.200001,0.200006,0.206131,0.200392,1.354482,0.211527,0.885570,0.203641,0.200004,0.893740,0.202013,0.236498,0.202997,0.832754,0.203599,0.201050,0.932835,0.932891,0.201502,157.241314,37.649196,7.388154,0.200009,0.201384,1.583922,0.804817,12.796279,0.204589,0.200002,0.205190,...,0.208830,0.204815,0.201126,1.599861,0.201490,0.201988,0.200001,0.953880,0.597579,1.257323,0.205273,0.200003,10.141555,0.200424,0.200716,0.200007,0.206440,0.200003,0.200003,1.703539,3.639146,1.168376,0.201700,0.201296,0.202057,0.201707,1.935609,1.327962,0.201165,0.200769,0.200696,0.200728,0.200003,0.200026,0.205826,0.201719,0.201415,0.202241,0.200005,0.205600
Topic1,0.202722,0.860732,0.202183,0.201248,0.200388,0.202213,0.203910,0.200000,0.201337,0.210045,0.200000,0.200002,0.204182,0.200620,0.201989,0.209949,0.200000,0.201277,0.200002,0.200970,0.202874,0.200926,0.206786,0.200388,0.200618,0.200002,0.200000,0.200000,0.200619,76.513465,28.404309,5.775672,0.889434,0.201569,0.201082,0.207869,0.211352,0.203366,0.200001,0.214045,...,0.203700,0.203522,0.211816,0.202596,0.201872,0.202675,0.200842,0.201531,0.201401,0.200001,1.693419,0.202132,0.209005,0.201715,0.203163,0.201160,0.205663,0.205766,0.209601,0.201419,0.204797,0.201226,0.201034,0.203221,0.200985,0.202336,0.203547,0.211925,0.206653,0.201907,0.200995,0.201167,0.202838,0.200009,0.202592,0.201854,0.203084,0.203788,0.200002,0.207121
Topic2,0.202957,0.312731,1.038090,0.790041,0.203620,0.202927,0.210585,0.201033,0.205583,0.201906,0.201957,0.203181,4.143647,0.200861,0.204861,0.207986,0.201033,0.201756,0.200006,0.207453,0.207807,0.214882,0.204314,0.203620,0.201346,0.215698,0.201395,0.201394,0.200712,5.945002,29.319457,0.221993,0.200012,0.202454,0.205685,0.200004,0.229395,4.855828,0.789973,0.208045,...,0.205339,1.055080,0.201422,0.204422,0.200011,1.681465,1.969371,0.203036,0.204736,0.200006,0.227013,0.826059,0.216312,6.032677,0.243145,1.632669,15.181779,0.200757,0.204424,0.207716,0.204947,0.684022,1.911441,0.204508,2.108687,0.201279,0.200016,0.205047,0.201302,71.532624,7.118483,22.600919,1.249662,1.421579,0.200002,0.204007,1.715254,0.200003,0.201195,0.203603
Topic3,0.203283,42.588960,0.202692,0.200252,0.201725,0.215424,0.203735,0.200642,0.201985,0.295129,0.203257,1.521080,0.205626,0.200531,0.205837,0.211038,0.200642,0.897397,0.202156,0.202820,0.201766,0.204616,1.317270,0.201725,0.201188,0.200006,0.210738,0.210683,0.201238,4.359898,22.002908,0.206570,0.202263,1.044243,0.201124,0.200890,0.213484,0.206279,0.200002,0.796833,...,7.057876,0.201874,3.121559,0.202651,0.202056,0.202077,0.200001,0.201040,1.128480,0.205359,0.217410,0.200003,0.205560,0.201206,0.203661,0.200008,0.253061,0.201891,1.597867,0.200002,0.474531,0.201607,0.201674,10.121956,0.204644,2.220619,0.200014,0.202035,3.618602,0.200864,0.200391,0.200557,0.201483,0.200032,1.897321,1.551052,0.206085,1.431518,0.990979,0.903103
Topic4,194.657608,0.204168,0.200689,0.202422,0.201266,1.331602,12.166730,0.200000,5.169123,0.207261,0.971120,0.208477,11.406263,1.589867,0.200873,15.564318,0.200000,16.920780,1.550152,0.200001,9.897778,9.214390,0.202739,0.201266,23.097960,1.403961,0.200546,0.200546,26.049801,366.821431,11.759886,0.205790,0.206121,0.200402,0.203837,0.200419,0.206800,0.200703,0.202279,0.209110,...,46.963477,0.202000,0.200861,0.200289,1.586843,0.201588,0.200473,0.201293,0.201849,0.206271,0.201519,0.201783,0.211509,0.201043,13.611076,0.206421,0.204146,0.818338,0.200760,0.201195,0.

In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=5):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=5)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4
Topic 0,fashion,women,health,like,magazine
Topic 1,magazine,great,articles,good,read
Topic 2,kindle,new,read,monthly,fire
Topic 3,national,geographic,food,issue,living
Topic 4,magazine,subscription,issue,year,amazon


In [ ]:
doc_complete = df['reviewText'][:25000]
doc_complete
#corpus = df["reviewText"].fillna(' ').values.tolist()
doc_complete1 = doc_complete.values.tolist()
print(doc_complete1[:10])
nltk.download('punkt')

['for computer enthusiast, MaxPC is a welcome sight in your mailbox. i can remember for years savorying every page of "boot" (as it was called in beginning) as i was (and still am) obcessed with PC\'s. Anyone, from advanced users - to beginners looking for knowledge - can profit from every issue of MaxPC. the icing on the cake is the subscription that comes with a CD-ROM as it is packed with demos, utilities, and other useful apps (very helpful for those not blessed with broadband connections). Until I discovered the community of hardware enthusiast web sites, MaxPC, formerly "boot", was my only really informative source for computing news and articles. To this day, i consider my subscription to it worth more than 10 subscriptions to most other computing mags. I can\'t wait until they merge with DVD media and maybe end up offering more info on Divx codecs, encoding your own movies, and best bang for the buck audio and video equipment. Try a few issues (with CD)and you may get hooked...

True

In [ ]:
from nltk.corpus import brown

from nltk import sent_tokenize, word_tokenize
#str_corpus = str(corpus)
#str_corpus_0 = corpus[0]
str_corpus_1 = str(doc_complete1[:100])
print('Sentences for the word : kindle\n',[i for i in sent_tokenize(str_corpus_1) if "kindle" in word_tokenize(i)])
print(i)

Sentences for the word : kindle
 ['That was a relief!\\n\\nI also like the fact that the magazine deals with both computer hardware, and software - though the emphasis is on hardware.\\n\\nThere are also letters to the computer doctor who diagnoses and then gives good advice on a range of computer illnesses and hiccups.\\n\\nYou also get the latest news taking place in the tech world and on copyright issues.\\n\\nOverall It is a well rounded publication with a little for everyone, young and old, great and small.\\n\\nI have the kindle for Ipad version, and it is the absolute best.']
The English Garden is a wonderful gardening magizine for serious gardeners. It has great practical advice, and good articles. Also, the photography is very good, makes you believe you might be able to do some of this. A great source for gardening supplies.


In [ ]:
str_corpus_1 = str(doc_complete1[:100])
print('Sentences for the word : subscription',[i for i in sent_tokenize(str_corpus_1) if "subscription" in word_tokenize(i)])

Sentences for the word : subscription ['the icing on the cake is the subscription that comes with a CD-ROM as it is packed with demos, utilities, and other useful apps (very helpful for those not blessed with broadband connections).', 'To this day, i consider my subscription to it worth more than 10 subscriptions to most other computing mags.', 'Maximum PC covers much more hardware and digs deeper.\\n\\nYou can subscribe (puchase) the magazine by going to [...]  1 year subscription ( 12 issues) is $19.95.', '\', \'I started off in the early days with a "Boot" subscription that eventually morphed itself into Maximum PC.', 'Maximum PC, PC Gamer and Computer Shopper were required reading for me in my early years of computer building and upgrading.\\n\\nFast forward many years later and I finally let me subscription lapse and never renewed.', "I've been reading Max PC for years, and now with my subscription I'm saving money on the mag,and from having to run around from store to store to ge

In [ ]:
str_corpus_1 = str(doc_complete1[:100])
print('Sentences for the word : article',[i for i in sent_tokenize(str_corpus_1) if "article" in word_tokenize(i)])

Sentences for the word : article ['Each title points out exactly what the article covers, leaving nothing to the imagination.\\n\\nThe writing in Antiques is certainly good and well- edited.', 'A web address or toll-free number to order the product follows the article.', 'I love the "How To 101" series they have in the magazine every month, it\\\'s so handy and it teaches you everything about whatever that month\\\'s article is about.', "Inside, there is an article written by celeb makeup artist, Kevyn Aucoin, who always has great advice for us on how to makeup in everyday life, as well as tell us about the glamorous world he\\'s in.", 'Their only saving grace is that they are so short (which is a problem when you run across the occasionally well-done, interesting article).', 'I would cancel it if I had a chance of getting my money back.\\n\\nUpdate:  The issues have become slightly more inclusive lately, with an article or two per issue not intended for the gaming community (i.e.']
